In [1]:
from IPython.core.display import display, HTML
import datetime
def info(str_):
    print(f'{datetime.datetime.now()} [ \033[1;94mINFO\x1b[0m  ] {str_}')
def ok(str_):
    print(f'{datetime.datetime.now()} [  \033[1;92mOK\x1b[0m   ] {str_}')
def warning(str_):
    print(f'{datetime.datetime.now()} [\x1b[1;31mWARNING\x1b[0m] {str_}')
def html(str_=''):
    display(HTML(str_))

In [2]:
import os
DP_examples_dirpath = '/home/ec2-user/SageMaker/mariano/notebooks/04. Model of DP/DP-relevant articles/'
assert os.path.exists(DP_examples_dirpath)
files = []
for dirpath, dirnames, filenames in os.walk(DP_examples_dirpath):
    for filename in filenames:
        filepath = os.path.join(dirpath,filename)
        assert os.path.isfile(filepath)
        files.append(filepath)
info(f'Number of files retrieved: {len(files)}')

2022-02-17 20:37:04.935627 [ INFO  ] Number of files retrieved: 11


In [3]:
import re
total_count=0
urls = []
ids = []
for file_ in files:
    content = open(file_, 'r').read()
    urls += re.findall('url = {(.*)}',content)
    ids += re.findall('/docview/([^/]*)/',content)
    total_count += len(urls)
    assert len(ids)==len(urls)

relevant_ids = set(ids)
print(len(urls))
print(len(relevant_ids))

538
535


In [4]:
import pandas as pd
from lxml import etree
from bs4 import BeautifulSoup

def get_date(filename):
    tree = etree.parse(filename)
    root = tree.getroot()
#     title = root.find('.//Title').text
    date = root.find('.//NumericDate').text
#     publisher = root.find('.//PublisherName').text
    assert date is not None
    
    return date

def get_title_and_text(filename):
    tree = etree.parse(filename)
    root = tree.getroot()
    if root.find('.//HiddenText') is not None:
        text = (root.find('.//HiddenText').text)

    elif root.find('.//Text') is not None:
        text = (root.find('.//Text').text)

    else:
        text = None
                       
    title = root.find('.//Title')
    if title is not None:
        title = title.text
    if not text is None:
        text = BeautifulSoup(text, parser='html.parser').get_text()

    return title,text

In [5]:
TS_dirpath = '/home/ec2-user/SageMaker/data/Toronto_Star_Publication_with_query/'
GM_dirpath = '/home/ec2-user/SageMaker/data/The_Globe_and_Mail_with_DP_filter_by_article_type/'
all_files = [TS_dirpath+file_id for file_id in os.listdir(TS_dirpath)]
all_files += [GM_dirpath+file_id for file_id in os.listdir(GM_dirpath)]

info(f'len(all_files):       {len(all_files):,}')

2022-02-17 20:37:12.216095 [ INFO  ] len(all_files):       192,433


In [6]:
import pandas as pd
df = pd.DataFrame(
                  ['unknown']*len(all_files), 
                  columns=['label']
                 )
df['id'] = [file_id.split('/')[-1][:-4] for file_id in all_files]
df['source']='GM'
df.iloc[:len(os.listdir(TS_dirpath)),-1]='TS'
df

,label,id,source
0,unknown,439574133,TS
1,unknown,435978513,TS
2,unknown,436354002,TS
3,unknown,435713287,TS
4,unknown,1818139138,TS
...,...,...,...
192428,unknown,1239399432,GM
192429,unknown,1368761450,GM
192430,unknown,1287302855,GM
192431,unknown,1242354607,GM


In [7]:
import numpy as np

relevants=0
dates = []

for idx,file_ in enumerate(all_files):
    id_ = file_.split('/')[-1][:-4]
    date = get_date(file_)
    year = int(date[:4])
    if id_ in relevant_ids:
        df.iloc[idx,0]='relevant'
        relevants+=1
    elif 1945 <=year and year<=1967 and df.iloc[idx,2]=='GM':
        df.iloc[idx,0]='irrelevant'
        
    dates.append(date)
df['date']=dates
info(f"Number of relevants:   {np.sum(df['label']=='relevant'):,}")
info(f"Number of irrelevants: {np.sum(df['label']=='irrelevant'):,}")
info(f"'Number of unknown:    {np.sum(df['label']=='unknown'):,}")
df

2022-02-17 20:39:08.495084 [ INFO  ] Number of relevants:   518
2022-02-17 20:39:08.506131 [ INFO  ] Number of irrelevants: 27,823
2022-02-17 20:39:08.516108 [ INFO  ] 'Number of unknown:    164,092


,label,id,source,date
0,unknown,439574133,TS,2009-04-13
1,unknown,435978513,TS,1989-07-30
2,unknown,436354002,TS,1991-02-22
3,unknown,435713287,TS,1988-01-22
4,unknown,1818139138,TS,2016-09-11
...,...,...,...,...
192428,unknown,1239399432,GM,1976-07-27
192429,unknown,1368761450,GM,2004-05-17
192430,irrelevant,1287302855,GM,1953-01-14
192431,unknown,1242354607,GM,1968-10-14


### SVM with average GloVe 

### BALANCED X

In [15]:
import numpy as np

rng = np.random.default_rng(2022)
relevant_idx = [idx  for idx in range(len(df)) if df.iloc[idx,0]=='relevant'] 
irrelevant_idx = [idx  for idx in range(len(df)) if df.iloc[idx,0]=='irrelevant'] 

irrelevant_selected_idx = rng.choice(irrelevant_idx, size=len(relevant_idx), replace=False)

print(len(irrelevant_selected_idx))

518


In [16]:


import spacy
nlp = spacy.load('en_core_web_lg', disable=['textcat','lemmatizer', 'parser', 'tagger','ner'])

example_no = len(df["label"]!='unknown')

X_balanced = np.zeros(shape=(len(irrelevant_selected_idx)*2, 600), dtype='float32')
y_balanced = np.zeros(shape=(len(irrelevant_selected_idx)*2,), dtype='int32')


selected_idx = set(irrelevant_selected_idx)
selected_idx = selected_idx.union(set(relevant_idx))

inst_no=0
for idx in range(len(df)):
    if idx in selected_idx:
        label, id_, date, source = df.iloc[idx,:]
        if source=='TS':
            file_ = TS_dirpath+id_+'.xml'
        else:
            file_ = GM_dirpath+id_+'.xml'
        assert label!='unknown'
#         if label!='unknown':
        title, text = get_title_and_text(file_)
        X_balanced[inst_no,:300] = nlp(title).vector
        X_balanced[inst_no,300:] = nlp(text).vector
        y_balanced[inst_no] = 1 if label=='relevant' else 0
        inst_no+=1

from sklearn.model_selection import cross_validate
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
parameters = {'kernel':('linear','rbf','poly'), 'C':[0.1,1,5,6,7,9,10,11,13,14,15,20]}
svc = SVC()
clf = GridSearchCV(svc, parameters,scoring='f1', cv=5,verbose=4)

results = clf.fit(X_balanced,y_balanced)
pd.DataFrame(results.cv_results_)

Fitting 5 folds for each of 36 candidates, totalling 180 fits
[CV 1/5] END ..............C=0.1, kernel=linear;, score=0.833 total time=   0.1s
[CV 2/5] END ..............C=0.1, kernel=linear;, score=0.841 total time=   0.1s
[CV 3/5] END ..............C=0.1, kernel=linear;, score=0.818 total time=   0.1s
[CV 4/5] END ..............C=0.1, kernel=linear;, score=0.760 total time=   0.1s
[CV 5/5] END ..............C=0.1, kernel=linear;, score=0.812 total time=   0.1s
[CV 1/5] END .................C=0.1, kernel=rbf;, score=0.776 total time=   0.2s
[CV 2/5] END .................C=0.1, kernel=rbf;, score=0.763 total time=   0.2s
[CV 3/5] END .................C=0.1, kernel=rbf;, score=0.772 total time=   0.2s
[CV 4/5] END .................C=0.1, kernel=rbf;, score=0.783 total time=   0.2s
[CV 5/5] END .................C=0.1, kernel=rbf;, score=0.774 total time=   0.2s
[CV 1/5] END ................C=0.1, kernel=poly;, score=0.776 total time=   0.2s
[CV 2/5] END ................C=0.1, kernel=poly

[CV 2/5] END .................C=10, kernel=poly;, score=0.811 total time=   0.2s
[CV 3/5] END .................C=10, kernel=poly;, score=0.766 total time=   0.1s
[CV 4/5] END .................C=10, kernel=poly;, score=0.796 total time=   0.1s
[CV 5/5] END .................C=10, kernel=poly;, score=0.816 total time=   0.2s
[CV 1/5] END ...............C=11, kernel=linear;, score=0.825 total time=   0.1s
[CV 2/5] END ...............C=11, kernel=linear;, score=0.779 total time=   0.1s
[CV 3/5] END ...............C=11, kernel=linear;, score=0.777 total time=   0.1s
[CV 4/5] END ...............C=11, kernel=linear;, score=0.768 total time=   0.1s
[CV 5/5] END ...............C=11, kernel=linear;, score=0.784 total time=   0.2s
[CV 1/5] END ..................C=11, kernel=rbf;, score=0.867 total time=   0.2s
[CV 2/5] END ..................C=11, kernel=rbf;, score=0.835 total time=   0.2s
[CV 3/5] END ..................C=11, kernel=rbf;, score=0.783 total time=   0.2s
[CV 4/5] END ...............

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.100823,0.001705,0.022292,0.000316,0.1,linear,"{'C': 0.1, 'kernel': 'linear'}",0.832536,0.840580,0.818182,0.759615,0.811594,0.812501,0.028354,12
1,0.153986,0.001550,0.062946,0.001413,0.1,rbf,"{'C': 0.1, 'kernel': 'rbf'}",0.776119,0.762963,0.771536,0.783270,0.773946,0.773567,0.006594,36
2,0.135845,0.001458,0.031572,0.000780,0.1,poly,"{'C': 0.1, 'kernel': 'poly'}",0.776119,0.774436,0.771536,0.781609,0.775194,0.775779,0.003294,35
3,0.093338,0.003292,0.016669,0.000708,1,linear,"{'C': 1, 'kernel': 'linear'}",0.834951,0.801887,0.800000,0.771429,0.811881,0.804030,0.020501,21
4,0.125346,0.003261,0.051131,0.001676,1,rbf,"{'C': 1, 'kernel': 'rbf'}",0.826291,0.818605,0.790909,0.781395,0.787037,0.800847,0.018059,24
5,0.112097,0.002151,0.024766,0.000438,1,poly,"{'C': 1, 'kernel': 'poly'}",0.833333,0.822430,0.814815,0.783410,0.809524,0.812702,0.016690,10
6,0.112284,0.005063,0.013691,0.000791,5,linear,"{'C': 5, 'kernel': 'linear'}",0.843137,0.788177,0.775120,0.788732,0.764706,0.791974,0.027093,25
7,0.119473,0.001101,0.043628,0.001208,5,rbf,"{'C': 5, 'kernel': 'rbf'}",0.857143,0.821256,0.786730,0.805687,0.807882,0.815740,0.023448,9
8,0.106334,0.002765,0.022556,0.000616,5,poly,"{'C': 5, 'kernel': 'poly'}",0.861244,0.822430,0.783019,0.792453,0.847291,0.821287,0.030232,6
9,0.116177,0.007584,0.013741,0.000961,6,linear,"{'C': 6, 'kernel': 'linear'}",0.839024,0.774510,0.772947,0.788732,0.770732,0.789189,0.025705,27


In [17]:
C = 7
kernel = 'rbf'

In [83]:
# # clf = SVC()
# from sklearn.naive_bayes import MultinomialNB
# cross_validate(SVC(),X, y, cv=5, scoring=['accuracy', 'precision', 'recall', 'f1'])

{'fit_time': array([0.21409798, 0.20563221, 0.24979544, 0.20647931, 0.14454699]),
 'score_time': array([0.13254738, 0.10718918, 0.06408405, 0.07168198, 0.05435395]),
 'test_accuracy': array([0.82211538, 0.8115942 , 0.77777778, 0.77294686, 0.77777778]),
 'test_precision': array([0.80733945, 0.78571429, 0.74358974, 0.75675676, 0.75892857]),
 'test_recall': array([0.84615385, 0.85436893, 0.84466019, 0.80769231, 0.81730769]),
 'test_f1': array([0.82629108, 0.81860465, 0.79090909, 0.78139535, 0.78703704])}

### GENERAL X (NOT-BALANCED)


In [8]:
import spacy
nlp = spacy.load('en_core_web_lg', disable=['textcat','lemmatizer', 'parser', 'tagger','ner'])

example_no = np.sum(df["label"]!='unknown')

X = np.zeros(shape=(example_no, 600), dtype='float32')
y = np.zeros(shape=(example_no,), dtype='int32')

inst_no=0
for idx in range(len(df)):
    label, id_, date, source = df.iloc[idx,:]
    if source=='TS':
        file_ = TS_dirpath+id_+'.xml'
    else:
        file_ = GM_dirpath+id_+'.xml'
    if label!='unknown':
        title, text = get_title_and_text(file_)
        X[inst_no,:300] = nlp(title).vector
        X[inst_no,300:] = nlp(text).vector
        y[inst_no] = 1 if label=='relevant' else 0
        inst_no+=1





# parameters = {'kernel':('linear','rbf'), 'C':[0.1,1,5,10]}
# svc = SVC()
# clf = GridSearchCV(svc, parameters,scoring='f1', cv=5,verbose=4,n_jobs=4)

# clf.fit(X,y)
# scores = cross_validate(MultinomialNB(),X,y,cv=5, scoring=['accuracy', 'precision', 'recall', 'f1'])


In [9]:
import pickle
pickle.dump(X, open('cache/X.p','wb'))
pickle.dump(y, open('cache/y.p','wb'))

In [ ]:
example_no = np.sum(df["label"]!='unknown')
example_no

In [ ]:
X[:]

In [19]:
from sklearn.metrics import confusion_matrix

clf = SVC(C=7, kernel='rbf')
clf.fit(X_balanced, y_balanced)

yhat = clf.predict(X)

m = confusion_matrix(y, yhat, labels=[0, 1])

tn,fp, fn, tp = m.ravel()

print(tn)
print(fp)
print(fn)
print(tp)

m

186991
4924
6
512


array([[186991,   4924],
       [     6,    512]])

In [ ]:
%%time

clf = SVC(C=7, kernel='rbf')
clf.fit(X, y)

yhat = clf.predict(X)

m = confusion_matrix(y, yhat, labels=[0, 1])

tn,fp, fn, tp = m.ravel()

print(tn)
print(fp)
print(fn)
print(tp)

m

In [58]:
# import spacy
# nlp = spacy.load('en_core_web_lg', disable=['textcat','lemmatizer', 'parser', 'tagger','ner'])

# example_no = len(df["label"]!='unknown')

# X = np.zeros(shape=(example_no, 600), dtype='float32')
# y = np.zeros(shape=(example_no,), dtype='int32')

# inst_no=0
# for idx in range(len(df)):
#     label, id_, date, source = df.iloc[idx,:]
#     if source=='TS':
#         file_ = TS_dirpath+id_+'.xml'
#     else:
#         file_ = GM_dirpath+id_+'.xml'
#     if label!='unknown':
#         title, text = get_title_and_text(file_)
#         X[inst_no,:300] = nlp(title).vector
#         X[inst_no,300:] = nlp(text).vector
#         y[inst_no] = 1 if label=='relevant' else 0
#         inst_no+=1


In [ ]:
# from sklearn.model_selection import cross_validate
# from sklearn.svm import SVC
# from sklearn.model_selection import GridSearchCV
# parameters = {'kernel':('linear','rbf'), 'C':[0.1,1,5,10]}
# svc = SVC()
# clf = GridSearchCV(svc, parameters,scoring='f1', cv=5,verbose=4,n_jobs=4)

# clf.fit(X,y)
# # scores = cross_validate(MultinomialNB(),X,y,cv=5, scoring=['accuracy', 'precision', 'recall', 'f1'])


In [ ]:
# import pickle
# pickle.dump

In [69]:
# # clf = SVC()
# from sklearn.naive_bayes import MultinomialNB
# cross_validate(SVC(),X[:10000,:],y[:10000], cv=5, scoring=['accuracy', 'precision', 'recall', 'f1'])

/home/ec2-user/SageMaker/.conda/envs/imm/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ec2-user/SageMaker/.conda/envs/imm/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ec2-user/SageMaker/.conda/envs/imm/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ec2-user/SageMaker/.conda/envs/imm

{'fit_time': array([3.66991735, 2.74251676, 2.52373266, 2.35394073, 2.34794617]),
 'score_time': array([0.49909353, 0.60177732, 0.53331375, 0.51753235, 0.4653523 ]),
 'test_accuracy': array([0.984 , 0.984 , 0.984 , 0.9835, 0.983 ]),
 'test_precision': array([0., 0., 0., 0., 0.]),
 'test_recall': array([0., 0., 0., 0., 0.]),
 'test_f1': array([0., 0., 0., 0., 0.])}

In [71]:
# np.sum(y[:10000]==1)

162

In [70]:
# import multiprocessing
# multiprocessing.cpu_count()

4

In [59]:
# print(X)

[[-0.22601451  0.01685     0.375      ... -0.04291409  0.0006775
   0.05622865]
 [-0.27549684  0.22640018 -0.1589972  ... -0.08522426 -0.00814553
   0.04669653]
 [-0.05172683  0.32064185  0.03239384 ... -0.03891717 -0.05615218
   0.03476123]
 ...
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]]


In [27]:
print(in_date)
print(not_in_date)
print(not_in_date_rel)

28328
164105
13


In [25]:
import numpy as np
np.sort(df["date"])

array(['1844-05-08', '1844-06-11', '1844-06-18', ..., '2022-01-16',
       '2022-01-16', '2022-01-16'], dtype=object)

In [13]:
title

'MPP looks from here to eternity'

In [32]:
print(len(df[df['label']=='relevant']))
print(len(df[df['label']=='irrelevant']))
print(len(df[df['label']=='unknown']))

505
27823
164105


In [34]:
print(len(df[df['label']=='relevant']))
print(len(df[df['label']=='irrelevant']))
print(len(df[df['label']=='unknown']))

518
27823
164092
